# Pandas - III - Data Ops 2 [Pivot]

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

pd.plotting.register_matplotlib_converters()
%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

# 1. Reshaping with Hierarchical Indices
Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:
 - <code>stack()</code> :This “rotates” or pivots from the columns in the data to the rows.
 - <code>unstack()</code> : This pivots from the rows into the columns.
I’ll illustrate these operations through a series of examples. Consider a small DataFrame with string arrays as row and column indexes:

In [2]:
data = pd.DataFrame(np.random.randint(1,100, 16).reshape((4, 4)), index = pd.Index([f'State {x}' for x in list('abcd'.upper())], name = 'State'), columns = pd.Index([f'Attr {y}' for y in range(1, 5)], name = 'Attributes'))
data.head()

Attributes,Attr 1,Attr 2,Attr 3,Attr 4
State,,,,
State A,52,61,21,19
State B,12,63,89,79
State C,41,87,30,5
State D,87,61,16,77


In [3]:
# Using the stack method on this data pivots the columns into the rows, producing a Series:
data2 = data.stack()
data2

State    Attributes
State A  Attr 1        52
         Attr 2        61
         Attr 3        21
         Attr 4        19
State B  Attr 1        12
         Attr 2        63
         Attr 3        89
         Attr 4        79
State C  Attr 1        41
         Attr 2        87
         Attr 3        30
         Attr 4         5
State D  Attr 1        87
         Attr 2        61
         Attr 3        16
         Attr 4        77
dtype: int32

In [4]:
# From a hierarchically indexed Series, you can rearrange the data back into a Data‐Frame with unstack:
data2 = data.stack()
data2.unstack()

Attributes,Attr 1,Attr 2,Attr 3,Attr 4
State,,,,
State A,52,61,21,19
State B,12,63,89,79
State C,41,87,30,5
State D,87,61,16,77


In [5]:
data2

State    Attributes
State A  Attr 1        52
         Attr 2        61
         Attr 3        21
         Attr 4        19
State B  Attr 1        12
         Attr 2        63
         Attr 3        89
         Attr 4        79
State C  Attr 1        41
         Attr 2        87
         Attr 3        30
         Attr 4         5
State D  Attr 1        87
         Attr 2        61
         Attr 3        16
         Attr 4        77
dtype: int32

In [6]:
# By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:
data2.unstack(0)

State,State A,State B,State C,State D
Attributes,,,,
Attr 1,52,12,41,87
Attr 2,61,63,87,61
Attr 3,21,89,30,16
Attr 4,19,79,5,77


In [7]:
data2.unstack('State')

State,State A,State B,State C,State D
Attributes,,,,
Attr 1,52,12,41,87
Attr 2,61,63,87,61
Attr 3,21,89,30,16
Attr 4,19,79,5,77


### Pandas Series Reference: 
 - 1. [Pandas Series doc](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Series.html)

In [8]:
# Unstacking might introduce missing data if all of the values in the level aren’t found in each of the subgroups:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2.index.names = ['key1', 'key2']
# pandas series object doesn't contain a 'column' attribute.
data2

key1  key2
one   a       0
      b       1
      c       2
      d       3
two   c       4
      d       5
      e       6
dtype: int64

In [9]:
data2.unstack()

key2,a,b,c,d,e
key1,,,,,
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [10]:
data2.unstack('key1')

key1,one,two
key2,,
a,0.0,NaN
b,1.0,NaN
c,2.0,4.0
d,3.0,5.0
e,NaN,6.0


In [11]:
# Stacking filters out missing data by default, so the operation is more easily invertible:
data2.unstack().stack()

key1  key2
one   a       0.0
      b       1.0
      c       2.0
      d       3.0
two   c       4.0
      d       5.0
      e       6.0
dtype: float64

In [12]:
# Keeping the missing values:
data2.unstack().stack(dropna=False)

key1  key2
one   a       0.0
      b       1.0
      c       2.0
      d       3.0
      e       NaN
two   a       NaN
      b       NaN
      c       4.0
      d       5.0
      e       6.0
dtype: float64

In [13]:
# When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:
series = data2
series

key1  key2
one   a       0
      b       1
      c       2
      d       3
two   c       4
      d       5
      e       6
dtype: int64

In [14]:
df = pd.DataFrame({'left': series, 'right': series + 5})
df

left  right
key1 key2             
one  a        0      5
     b        1      6
     c        2      7
     d        3      8
two  c        4      9
     d        5     10
     e        6     11

In [15]:
df = pd.DataFrame({'left': series, 'right': series + 5}, columns=pd.Index(['left', 'right'], name='side'))
df

side       left  right
key1 key2             
one  a        0      5
     b        1      6
     c        2      7
     d        3      8
two  c        4      9
     d        5     10
     e        6     11

In [16]:
df.unstack()

side left                     right                      
key2    a    b    c    d    e     a    b    c     d     e
key1                                                     
one   0.0  1.0  2.0  3.0  NaN   5.0  6.0  7.0   8.0   NaN
two   NaN  NaN  4.0  5.0  6.0   NaN  NaN  9.0  10.0  11.0

In [17]:
df.unstack('key1')

side left      right      
key1  one  two   one   two
key2                      
a     0.0  NaN   5.0   NaN
b     1.0  NaN   6.0   NaN
c     2.0  4.0   7.0   9.0
d     3.0  5.0   8.0  10.0
e     NaN  6.0   NaN  11.0

In [18]:
# When calling stack, we can indicate the name of the axis to stack:
df2 = df.unstack('key1')
df2.stack()

side       left  right
key2 key1             
a    one    0.0    5.0
b    one    1.0    6.0
c    one    2.0    7.0
     two    4.0    9.0
d    one    3.0    8.0
     two    5.0   10.0
e    two    6.0   11.0

In [19]:
df2.stack('side')

key1        one   two
key2 side            
a    left   0.0   NaN
     right  5.0   NaN
b    left   1.0   NaN
     right  6.0   NaN
c    left   2.0   4.0
     right  7.0   9.0
d    left   3.0   5.0
     right  8.0  10.0
e    left   NaN   6.0
     right  NaN  11.0

# 2. Pivot:
## 2.1 Pivot Table basics

In [20]:
# Load Example Data:
df = pd.read_excel("sales.xlsx")
df

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [21]:
# For convenience sake, let’s define the status column as a 'category' and set the order we want to view. This isn’t strictly required but helps us keep the order we want as we work through analyzing the data.
df['Status'] = df['Status'].astype('category') # Changing datatype
df['Status'].cat.set_categories(['won', 'pending', 'presented', 'declined'], inplace = True)

In [22]:
df['Name'].unique().tolist()

['Trantow-Barrows',
 'Fritsch, Russel and Anderson',
 'Kiehn-Spinka',
 'Kulas Inc',
 'Jerde-Hilpert',
 'Barton LLC',
 'Herman LLC',
 'Purdy-Kunde',
 'Stokes LLC',
 'Kassulke, Ondricka and Metz',
 'Keeling LLC',
 'Koepp Ltd']

In [23]:
df['Name'].value_counts()

Trantow-Barrows                 3
Koepp Ltd                       2
Stokes LLC                      2
Kulas Inc                       2
Keeling LLC                     1
Kiehn-Spinka                    1
Kassulke, Ondricka and Metz     1
Fritsch, Russel and Anderson    1
Purdy-Kunde                     1
Barton LLC                      1
Jerde-Hilpert                   1
Herman LLC                      1
Name: Name, dtype: int64

In [24]:
# The simplest pivot table must have a dataframe and an index . In this case, let’s use the 'Name' as our index.
pd.pivot_table(df, index = 'Name')

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


In [25]:
# You can have multiple indexes as well. In fact, most of the pivot_table args can take multiple values via a list.
pd.pivot_table(df, index = ['Name', 'Rep', 'Manager'])

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150,35000,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550,35000,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962,65000,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290,5000,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599,7000,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981,100000,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832,65000,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833,35000,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895,25000,1.500000


In [26]:
# This is interesting but not particularly useful. What we probably want to do is look at this by Manager and Rep. It’s easy enough to do by changing the index:
pd.pivot_table(df,index=['Manager', 'Rep'])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

You can see that the pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. Now we start to get a glimpse of what a pivot table can do for us.

For this purpose, the Account and Quantity columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the <code>values</code> field.

In [27]:
pd.pivot_table(df, index = ['Manager', 'Rep'], values = ['Price']).round(2)

Price
Manager       Rep                    
Debra Henley  Craig Booker   20000.00
              Daniel Hilton  38333.33
              John Smith     20000.00
Fred Anderson Cedric Moss    27500.00
              Wendy Yule     44250.00

In [28]:
# The price column automatically averages the data but we can do a count or a sum. 
# Adding them is simple using aggfunc and np.sum .

pd.pivot_table(df,index=['Manager','Rep'],values=['Price'],aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [29]:
# aggfunc can take a list of functions. Let’s try a mean using the numpy 'mean()' function and 'len()' to get a count.
pd.pivot_table(df,index=['Manager','Rep'],values=['Price'],aggfunc=[np.sum, np.mean, len]).round(2)

sum      mean   len
                              Price     Price Price
Manager       Rep                                  
Debra Henley  Craig Booker    80000  20000.00     4
              Daniel Hilton  115000  38333.33     3
              John Smith      40000  20000.00     2
Fred Anderson Cedric Moss    110000  27500.00     4
              Wendy Yule     177000  44250.00     4

If we want to see sales broken down by the products, the <code>columns</code> variable allows us to define one or more columns.
### Columns vs. Values
I think one of the confusing points with the <code>pivot_table</code> is the use of <code>columns</code> and <code>values</code> . Remember, <code>columns</code> are optional - they provide an additional way to segment the actual values you care about. The aggregation functions are applied to the <code>values</code> you list.

#### Rule of thumb:

if you have a problem statement which says "FOR EACH (X), I WANT (Y), PER (Z)" then lets try to fit one of the examples explained in the article to this query.

FOR EACH X=MANAGER I WANT Y=QUANTIY(the sum/average/count whatever) PER Z=PRODUCT.

Now everything under X goes to INDEXS, everything under Y goes to VALUES and everything under Z goes to COLUMNS.

The aggregation function is pretty straight forward and you don't have to worry much to remember it. Just remember that they operate in tandem with VALUES only.

The above query can further be expanded to as below:
FOR EACH X1 and X2, I WANT Y1 and Y2, PER Z1 and Z2 (if you have more than 1 attributes to consider and I guess there is no limit to this depth).

The beauty is you can swap X1, with Y1 or Y1 with Z1 or X2 with Z2 etc, but only few might give you meaningful results.

In [30]:
df2 = pd.pivot_table(df,index=['Manager','Rep'], columns = ['Product'], values=['Price'], aggfunc=[np.sum, np.mean]).round(2)
df2

sum                                  mean  \
                                Price                                 Price   
Product                           CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                             
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0  32500.0   
              Daniel Hilton  105000.0         NaN     NaN  10000.0  52500.0   
              John Smith      35000.0      5000.0     NaN      NaN  35000.0   
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0  47500.0   
              Wendy Yule     165000.0      7000.0  5000.0      NaN  82500.0   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker       5000.0     NaN  10000.0  
              Daniel Hilton         NaN     NaN  10000.0  
              John Smith         5000.0     NaN      NaN  
Fred Anderson Cedric Moss        5000.0     NaN  10000.0  
              Wendy Yule         7000.0  5000.0      NaN

In [31]:
# The NaN’s are a bit distracting. If we want to remove them, we could use fill_value to set them to 0.
df2 = pd.pivot_table(df,index=['Manager','Rep'], columns = ['Product'], values=['Price'], aggfunc=[np.sum, np.mean], fill_value = 0).round(2)
df2

sum                                mean  \
                              Price                               Price   
Product                         CPU Maintenance Monitor Software    CPU   
Manager       Rep                                                         
Debra Henley  Craig Booker    65000        5000       0    10000  32500   
              Daniel Hilton  105000           0       0    10000  52500   
              John Smith      35000        5000       0        0  35000   
Fred Anderson Cedric Moss     95000        5000       0    10000  47500   
              Wendy Yule     165000        7000    5000        0  82500   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker         5000       0    10000  
              Daniel Hilton           0       0    10000  
              John Smith           5000       0        0  
Fred Anderson Cedric Moss          5000       0    10000  
              Wendy Yule           7000    5000        0

In [32]:
# I think it would be useful to add the quantity as well. Add Quantity to the values list.
df3 = pd.pivot_table(df,index=['Manager','Rep'], columns = ['Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value = 0).round(2)
df3

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [33]:
# What’s interesting is that you can move items to the index to get a different visual representation. Remove Product from the columns and add to the index .
pd.pivot_table(df,index=['Manager','Rep', 'Product'], values=['Price', 'Quantity'], aggfunc=[np.sum], fill_value = 0).round(2)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [34]:
# For this data set, this representation makes more sense. Now, what if I want to see some totals? margins=True does that for us.
pd.pivot_table(df,index=['Manager','Rep', 'Product'], values=['Price', 'Quantity'], aggfunc=[np.sum, np.mean], fill_value = 0, margins = True).round(2)

sum            mean         
                                          Price Quantity  Price Quantity
Manager       Rep           Product                                     
Debra Henley  Craig Booker  CPU           65000        2  32500     1.00
                            Maintenance    5000        2   5000     2.00
                            Software      10000        1  10000     1.00
              Daniel Hilton CPU          105000        4  52500     2.00
                            Software      10000        1  10000     1.00
              John Smith    CPU           35000        1  35000     1.00
                            Maintenance    5000        2   5000     2.00
Fred Anderson Cedric Moss   CPU           95000        3  47500     1.50
                            Maintenance    5000        1   5000     1.00
                            Software      10000        1  10000     1.00
              Wendy Yule    CPU          165000        7  82500     3.50
                            Maintenance    7000        3   7000     3.00
                            Monitor        5000        2   5000     2.00
All                                      522000       30  30705     1.76

In [35]:
# Let’s move the analysis up a level and look at our pipeline at the manager level. Notice how the status is ordered based on our earlier category definition.
pd.pivot_table(df, index = ['Manager', 'Status'], values = ['Price', 'Quantity'], aggfunc = [np.sum, np.mean], fill_value = 0, margins = True).round(2)

sum               mean         
                          Price Quantity     Price Quantity
Manager       Status                                       
Debra Henley  won         65000        2  65000.00     2.00
              pending     50000        6  16666.67     2.00
              presented   50000        3  16666.67     1.00
              declined    70000        2  35000.00     1.00
Fred Anderson won        172000       10  57333.33     3.33
              pending      5000        1   5000.00     1.00
              presented   45000        4  15000.00     1.33
              declined    65000        2  65000.00     2.00
All                      522000       30  30705.88     1.76

In [36]:
# A really handy feature is the ability to pass a dictionary to the aggfunc so you can perform different functions on each of the values you select. This has a side-effect of making the labels a little cleaner.
pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  won         65000           0       0        0        1   
              pending     40000       10000       0        0        1   
              presented   30000           0       0    20000        1   
              declined    70000           0       0        0        2   
Fred Anderson won        165000        7000       0        0        2   
              pending         0        5000       0        0        0   
              presented   30000           0    5000    10000        1   
              declined    65000           0       0        0        1   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending             2       0        0  
              presented           0       0        2  
              declined            0       0        0  
Fred Anderson won                 1       0        0  
              pending             1       0        0  
              presented           0       1        1  
              declined            0       0        0

In [37]:
# You can provide a list of aggfunctions to apply to each value too:
table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

#### <font color = 'blue'>When to use pivot tables?</font>
My general rule of thumb is that once you use multiple grouby you should evaluate whether a pivot table is a useful approach.

## 2.2 Advanced Pivot Table Filtering
Once you have generated your data, it is in a <code>DataFrame</code> so you can filter on it using your standard <code>DataFrame</code> functions.

In [38]:
# If you want to look at just one manager:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley won        65000           0       0        0  65000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             declined   35000           0       0        0  70000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley won             0        0        1           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             declined        0        0        2           0       0        0

In [39]:
# We can look at all of our pending and won deals.
table.query('Status == ["pending","won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0